## Filtering non-overlapping transcripts for metagene analysis

In [1]:
import pybedtools 
import sys
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

#### set script parameters


In [2]:
home_dir = '/home/salma/Coding/cluster' 

#interval: genomic features are sepaced by at least 'interval' base pairs 
interval = 700
clear_both_strands = False

#input files
TIF_input_file = os.path.join(home_dir, 'Annotations/steinmetz_transcripts_tifs_clean_sorted.gff')

#output files
TSS_nonoverlapping_file = os.path.join(home_dir, 'Annotations/salma/steinmetz_nonoverlapping_sense_TSS_' + str(interval) +'.gff')
pA_nonoverlapping_file = os.path.join(home_dir, 'Annotations/salma/steinmetz_nonoverlapping_sense_pA_' + str(interval) +'.gff')


In [3]:
tif = pd.read_table(TIF_input_file, sep='\t', header=None)
tif.columns = ['chr','ref','type','start','end','c1','strand','c2','gene_name']

expanded_tif = tif.copy()
expanded_tif.start = expanded_tif.start - int(interval/2)
expanded_tif.end = expanded_tif.end + int(interval/2)

In [4]:
bed_expanded = pybedtools.bedtool.BedTool.from_dataframe(expanded_tif)

def overlap_filter(feature):
    return int(feature[3]) > 1

bed_expanded_overlaps = bed_expanded.merge(c = 1, o = 'count').filter(overlap_filter)

non_overlapping = bed_expanded.subtract(bed_expanded_overlaps, A = True)

In [5]:
#non-overlapping on both TSS and pA
non_overlapping.count()

169

In [6]:
#too little left
#another strategy

#### filtering overlaps on TSS and pA individually (here on sense strand)

In [7]:
tif_bed = pybedtools.bedtool.BedTool.from_dataframe(tif)

In [8]:
upstream_TSS = list()
for i in range(len(tif)):
    if (tif.loc[i,'strand'] == '+'):
        upstream_TSS.append([tif.loc[i,'chr'],
                         '.','.',
                         tif.loc[i,'start']-interval,
                         tif.loc[i,'start']-1,
                         i,'+','.', '.'])
    else:
        upstream_TSS.append([tif.loc[i,'chr'],
                         '.','.',
                         tif.loc[i,'end'] + 1,
                         tif.loc[i,'end'] + interval,
                         i,'-','.', '.'])
upstream_TSS = pd.DataFrame(upstream_TSS, columns=tif.columns)
upstream_TSS_bed = pybedtools.bedtool.BedTool.from_dataframe(upstream_TSS)

In [9]:
TSS_nonoverlapping = upstream_TSS_bed.subtract(tif_bed, A = True, s = not clear_both_strands)

indices = TSS_nonoverlapping.to_dataframe().loc[:,'score']
TSS_nonoverlapping = tif.iloc[indices,:]
pybedtools.bedtool.BedTool.from_dataframe(TSS_nonoverlapping).saveas(TSS_nonoverlapping_file)
len(TSS_nonoverlapping)

3193

In [10]:
downstream_pA = list()
for i in range(len(tif)):
    if (tif.loc[i,'strand'] == '+'):
        downstream_pA.append([tif.loc[i,'chr'],
                         '.','.',
                         tif.loc[i,'end'] + 1,
                         tif.loc[i,'end'] + interval,
                         i,'+','.', '.'])
    else:
        downstream_pA.append([tif.loc[i,'chr'],
                         '.','.',
                         tif.loc[i,'start'] - interval,
                         tif.loc[i,'start'] - 1,
                         i,'-','.', '.'])
downstream_pA = pd.DataFrame(downstream_pA, columns=tif.columns)
downstream_pA_bed = pybedtools.bedtool.BedTool.from_dataframe(downstream_pA)

In [11]:
pA_nonoverlapping = downstream_pA_bed.subtract(tif_bed, A = True, s = not clear_both_strands)

indices = pA_nonoverlapping.to_dataframe().loc[:,'score']
pA_nonoverlapping = tif.iloc[indices,:]
pybedtools.bedtool.BedTool.from_dataframe(pA_nonoverlapping).saveas(pA_nonoverlapping_file)
len(pA_nonoverlapping)

3193